In [1]:
import pickle
from dataclasses import fields
from pathlib import Path

import igl
import numpy as np
import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import parameterization as par

In [2]:
patient_id = "01"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

with segmentation_file_name.open("rb") as f:
    objects_to_load = pickle.load(f)
boundary_paths, connection_paths, markers, parameterized_paths = objects_to_load

In [3]:
uac_config = par.UACConfig(
    alpha_min=0,
    alpha_max=1,
    beta_min=0,
    beta_max=1,
    radius_pv_inner=0.1,
    radius_pv_outer=0.05,
    radius_laa=0.1,
)

uac_specs = par.get_uac_specs(uac_config)
uac_paths = par.compute_uac_paths(parameterized_paths, uac_specs)

In [4]:
uac_to_visualize = "beta"

plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    mesh,
    color="lightgrey",
    show_edges=True,
    edge_opacity=0.3,
)
for uac_path_field in fields(uac_paths):
    uac_path = getattr(uac_paths, uac_path_field.name)
    uac_path_mesh = pv.PolyData(mesh.points[uac_path.inds])
    uac_path_mesh.point_data["alpha"] = uac_path.alpha
    uac_path_mesh.point_data["beta"] = uac_path.beta
    plotter.add_points(
        uac_path_mesh,
        scalars=uac_to_visualize,
        point_size=5,
        render_points_as_spheres=True,
    )
plotter.show()

Widget(value='<iframe src="http://localhost:37067/index.html?ui=P_0x7fb62b0cfa10_0&reconnect=auto" class="pyvi…

In [17]:
vertices = mesh.points
simplices = mesh.faces.reshape(-1, 4)[:, 1:4]
boundary_inds, boundary_vals = par.extract_boundary_data(uac_paths.MV)
harmonic_mapping = igl.harmonic(vertices, simplices, boundary_inds, boundary_vals, 1)
harmonic_mapping = np.hstack([harmonic_mapping, np.zeros((vertices.shape[0], 1))])
harmonic_mesh = pv.PolyData.from_regular_faces(harmonic_mapping, simplices)

In [6]:
plotter = pv.Plotter(window_size=[800, 800])
plotter.add_mesh(
    harmonic_mesh,
    color="lightgrey",
    style="wireframe",
)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:37067/index.html?ui=P_0x7fb618dad090_1&reconnect=auto" class="pyvi…

In [20]:
new_boundary_inds, new_boundary_vals = par.extract_boundary_data(
    uac_paths.MV,
    uac_paths.LIPV_outer,
    uac_paths.LSPV_outer,
    uac_paths.RIPV_outer,
    uac_paths.RSPV_outer,
)

slim_data = igl.slim_precompute(
    V=np.array(harmonic_mesh.points),
    F=harmonic_mesh.faces.reshape(-1, 4)[:, 1:],
    V_init=np.array(harmonic_mesh.points)[:, :2],
    slim_energy=igl.MappingEnergyType.CONFORMAL,
    b=new_boundary_inds,
    bc=new_boundary_vals,
    soft_p=1e5,
)
slim_mapping = igl.slim_solve(data=slim_data, iter_num=10)

In [21]:
slim_mapping = np.hstack([slim_mapping, np.zeros((vertices.shape[0], 1))])
slim_mesh = pv.PolyData.from_regular_faces(slim_mapping, simplices)

plotter = pv.Plotter(window_size=[800, 800])
plotter.add_mesh(
    slim_mesh,
    color="lightgrey",
    style="wireframe",
)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:37067/index.html?ui=P_0x7fb618daf110_6&reconnect=auto" class="pyvi…